In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
file_id = None #'<add_you_coco_dataset_dive_link>'  #Eg. '1fdgdgssytsyusrtyshgg' ensure data to be zip
file_id_weigths = None #'<trained_weights any>' #Eg. '1fdgdgsgsjusysetg' : format: model_weights_00_100.pth

In [2]:
import gdown
import shutil
from pathlib import Path
Gdrive = False # IF you want to save data of images keep true
folder_path = '/content/drive/MyDrive/' if Gdrive else '/content/'

In [4]:
%%bash
cd {folder_path} #goto drive location
git clone https://github.com/omer770/Fine_Tune_Grounding_Dino.git
cd Fine_Tune_Grounding_Dino
cd weights
wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth #default weights
cd /content/

bash: line 1: cd: {folder_path}: No such file or directory
Cloning into 'Fine_Tune_Grounding_Dino'...


In [ ]:
if file_id:
  zip_path = gdown.download(id=file_id)
  !unzip -q {zip_path} -d {folder_path+'Fine_Tune_Grounding_Dino/data'}
if file_id_weigths:
  weights_path = gdown.download(id=file_id_weigths)
  weights_Dir = Path('/content/drive/MyDrive/Colab_zip/GroundingDINO/weights')
  weights_Dir.mkdir(parents=True, exist_ok=True)
  shutil.move(weights_path, weights_Dir)

In [4]:
%pwd

'/content'

Run at Fine_Tune_Grounding_Dino directory

In [3]:
%cd {folder_path+'Fine_Tune_Grounding_Dino'}

/content/Fine_Tune_Grounding_Dino


In [6]:
!pip install -r requirements.txt -q

ERROR: Operation cancelled by user


In [11]:
!pip install -e .

Obtaining file:///content/Fine_Tune_Grounding_Dino
  Preparing metadata (setup.py) ... done
  Running setup.py develop for groundingdino


In [4]:
from groundingdino.util.dataset import annotation_coco_2_pd_converter
from google.colab.patches import cv2_imshow
import cv2

In [5]:
list_class_to_train = ['pool']
path_2_images = folder_path+'/Fine_Tune_Grounding_Dino/data/images'
path_2_csv = folder_path+'Fine_Tune_Grounding_Dino/data/annotations/annotation.csv'
path_2_ann_json = folder_path+'Fine_Tune_Grounding_Dino/data/annotations/instances_default.json'
path_2_sub_csv = path_2_csv.replace('annotation.','annotation_sub.')
df_ann, df_all, df_shape = annotation_coco_2_pd_converter(list_class_to_train=list_class_to_train,path_2_ann_json = path_2_ann_json,path_2_images = path_2_images,
                                                  path_2_sub_csv=path_2_sub_csv,path_2_csv = path_2_csv)


In [6]:
df_all

,label_name,bbox_x,bbox_y,bbox_width,bbox_height,image_name,image_width,image_height
9631,caravan,390,290,155,85,61559_108220_18.jpg,1024,1024
25549,circular structure,0,368,61,132,61516_108164_18.jpg,1024,1024
18054,circular structure,1008,703,15,78,61566_108177_18.jpg,1024,1024
8504,pool,427,882,60,82,61562_108093_18.jpg,1024,1024
20589,pool,492,527,43,82,61530_108200_18.jpg,1024,1024
995,circular structure,583,352,73,71,61608_108186_18.jpg,1024,1024
10872,caravan,703,90,67,21,61598_108234_18.jpg,1024,1024
6134,caravan,687,222,64,20,61581_108088_18.jpg,1024,1024
12695,pool,479,854,98,65,61552_108093_18.jpg,1024,1024
7513,caravan,610,236,70,23,61575_108256_18.jpg,1024,1024


In [8]:
df_ann

,label_name,bbox_x,bbox_y,bbox_width,bbox_height,image_name,image_width,image_height
0,building,392,788,173,121,61614_108109_18.jpg,1024,1024
1,building,673,259,148,133,61614_108109_18.jpg,1024,1024
2,building,981,1007,39,16,61614_108109_18.jpg,1024,1024
3,building,494,519,116,123,61614_108109_18.jpg,1024,1024
4,building,866,136,157,111,61614_108109_18.jpg,1024,1024
5,building,760,0,32,15,61614_108109_18.jpg,1024,1024
6,building,591,832,26,27,61614_108109_18.jpg,1024,1024
7,building,916,205,107,139,61614_108109_18.jpg,1024,1024
8,building,0,831,146,130,61614_108109_18.jpg,1024,1024
9,building,29,909,161,114,61614_108109_18.jpg,1024,1024


In [15]:
!nvidia-smi

Fri Apr 19 06:53:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!python train_run.py

In [ ]:
!python test_run.py
cv2_imshow(cv2.imread('results/result.jpg'))

In [ ]:
#Test
import torch
import cv2
from test_run import process_image
from pathlib import Path
from groundingdino.util.inference import load_model, load_image, predict, annotate
from google.colab.patches import cv2_imshow

weights_Dir = Path('/content/drive/MyDrive/Colab_zip/GroundingDINO/weights')
filePaths = [file for file in weights_Dir.iterdir() if file.name.startswith('model_weights')]
try: model_weights= str(filePaths[-1])
except: model_weights=None
#IMAGE_PATH = '/content/drive/MyDrive/Colab_zip/GroundingDINO/data/test_images/img.jpg'
IMAGE_PATH = "/content/Fine_Tune_Grounding_Dino/data/test_images/img_test.jpg"
#IMAGE_PATH = "/content/Fine_Tune_Grounding_Dino/data/test_images/img.jpg"
#TEXT_PROMPT = "building . pool . carvan .circular stucture"
TEXT_PROMPT = "pool"
BOX_TRESHOLD = 0.30
TEXT_TRESHOLD = 0.30
print("choosen weights: ",model_weights)
try:
  process_image(model_weights=model_weights,
        image_path= IMAGE_PATH,
        text_prompt=TEXT_PROMPT,
        box_threshold=BOX_TRESHOLD,
        text_threshold=TEXT_TRESHOLD)
  cv2_imshow(cv2.imread('results/result.jpg'))
except:print(f"No Detections found of {TEXT_PROMPT} Category")